In [20]:
!pip install opendatasets

In [21]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/atifaliak/youtube-comments-dataset")

Skipping, found downloaded files in "./youtube-comments-dataset" (use force=True to force download)


In [22]:
import pandas as pd
import numpy as np

In [23]:
data=pd.read_csv("/content/youtube-comments-dataset/YoutubeCommentsDataSet.csv")

In [24]:
data.head()

,Comment,Sentiment
0,lets not forget that apple pay in 2014 require...,neutral
1,here in nz 50 of retailers don’t even have con...,negative
2,i will forever acknowledge this channel with t...,positive
3,whenever i go to a place that doesn’t take app...,negative
4,apple pay is so convenient secure and easy to ...,positive


In [25]:
data.head(5)

,Comment,Sentiment
0,lets not forget that apple pay in 2014 require...,neutral
1,here in nz 50 of retailers don’t even have con...,negative
2,i will forever acknowledge this channel with t...,positive
3,whenever i go to a place that doesn’t take app...,negative
4,apple pay is so convenient secure and easy to ...,positive


In [26]:
data.shape

(18408, 2)

In [27]:
data["Sentiment"].value_counts()

,count
Sentiment,
positive,11432
neutral,4638
negative,2338


In [28]:
data.isnull().sum()

,0
Comment,44
Sentiment,0


In [29]:
data["Comment"][0]

'lets not forget that apple pay in 2014 required a brand new iphone in order to use it a significant portion of apples user base wasnt able to use it even if they wanted to as each successive iphone incorporated the technology and older iphones were replaced the number of people who could use the technology increased'

In [43]:
!pip install contractions

In [44]:
import re
import nltk
import contractions
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [45]:
def text_processing(text):
  if pd.isna(text):
    return ""
  text=text.lower()
  text=re.sub(r"[^a-zA-z\s]"," ",text)
  text=contractions.fix(text)
  tokens=word_tokenize(text)
  stop_words=set(stopwords.words("english"))
  for word in tokens:
    if word in stop_words:
      tokens.remove(word)
  return " ".join(tokens)

data["Comment"]=data["Comment"].apply(text_processing)
print(data["Comment"])



0        let us forget apple pay required brand new iph...
1        nz retailers even contactless credit card mach...
2        forever acknowledge channel help lessons ideas...
3        whenever go place take apple pay happen often ...
4        apple pay convenient secure easy use used the ...
                               ...                        
18403    really like point engineering toolboxes think ...
18404    started exploring field really good reminder g...
18405    excelente video con una pregunta filos fica pr...
18406    hey daniel discovered channel couple days ago ...
18407    great focus key playful approach also speed th...
Name: Comment, Length: 18408, dtype: object


In [46]:
data["Sentiment"].replace({'positive':2,'neutral':1,'negative':0},inplace=True)

<ipython-input-46-d46d36c5746c>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["Sentiment"].replace({'positive':2,'neutral':1,'negative':0},inplace=True)


In [47]:
od.download("https://www.kaggle.com/datasets/adarshsng/googlenewsvectors")

Skipping, found downloaded files in "./googlenewsvectors" (use force=True to force download)


In [48]:
import gensim

# Load the Google’s pre-trained Word2Vec model
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format("/content/googlenewsvectors/GoogleNews-vectors-negative300.bin", binary=True)

In [81]:
import numpy as np

def get_word2vec_vector(comment, model):
    word_vectors = [model[word] for word in comment.split() if word in model]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(model.vector_size)

# Convert all comments to Word2Vec vectors
data["Word2Vec_Vector"] = data["Comment"].apply(lambda x: get_word2vec_vector(x, word2vec_model))


In [82]:
X = np.vstack(data["Word2Vec_Vector"].values)  # Features
y = data["Sentiment"].values

# Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [83]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Initialize the classifier
classifier1 = RandomForestClassifier()

# Train the model
classifier1.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier1.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7153720803910918
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.16      0.26       475
           1       0.67      0.47      0.55       937
           2       0.72      0.93      0.82      2270

    accuracy                           0.72      3682
   macro avg       0.73      0.52      0.54      3682
weighted avg       0.72      0.72      0.68      3682



In [84]:
# Train a Decision Tree classifier (You can replace it with Logistic Regression, Random Forest, etc.)
classifier2 = DecisionTreeClassifier()
classifier2.fit(X_train, y_train)

# Predict on test data
y_pred = classifier2.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6121673003802282
Classification Report:
               precision    recall  f1-score   support

           0       0.31      0.32      0.31       475
           1       0.44      0.44      0.44       937
           2       0.75      0.74      0.75      2270

    accuracy                           0.61      3682
   macro avg       0.50      0.50      0.50      3682
weighted avg       0.61      0.61      0.61      3682



In [85]:
# Train a Logistic Regression model
classifier3 = LogisticRegression()
classifier3.fit(X_train, y_train)

# Predict on test data
y_pred = classifier3.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7582835415535035
Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.44      0.51       475
           1       0.64      0.57      0.61       937
           2       0.82      0.90      0.86      2270

    accuracy                           0.76      3682
   macro avg       0.69      0.64      0.66      3682
weighted avg       0.75      0.76      0.75      3682



In [88]:
def predict_sentiment(model, classifier):
    comment = input("Enter a comment: ")  # Ask user for input
    comment_vector = get_word2vec_vector(comment, model).reshape(1, -1)  # Convert to vector
    sentiment_map = {2: "Positive", 1: "Neutral", 0: "Negative"}
    print("Predicted Sentiment:", sentiment_map[classifier.predict(comment_vector)[0]])

# Call the function for sentiment prediction
predict_sentiment(word2vec_model, classifier3)


Enter a comment: Wow! The editing and content are top-notch. Subscribed!
Predicted Sentiment: Positive
